In [2]:

#Loading required libraries.
import pandas as pd
import numpy as np
import re
import string
from pprint import pprint
from collections import Counter
import keras

from keras.layers import Embedding
import tensorflow as tf

from keras.datasets import imdb
from keras import preprocessing



#Loading nltk library and its packages for text processing.
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

#Downloading some necessary word lists.
#nltk.download('wordnet')
#nltk.download('stopwords')

#Downloading all dependencies and corpora.
#nltk.download()
#nltk.download('all', halt_on_error=False)

#Importing pattern and its dependencies.
#import pattern
#from pattern.en import tag
#from pattern.en import parse
#from pattern.en import pprint



In [3]:
#Reading Training and Testing Datasets.
raw_data = pd.read_csv("/content/drive/My Drive/Colab_Datasets/Tweets_Data/train.csv")
#test_data = pd.read_csv("C:/Users/Bhair/OneDrive - University of Oklahoma/Master_of_Science_Data/Data Science Practice/Twitter Sentiment Analysis/Data/tweet-sentiment-extraction/test.csv")



#Contraction directory.

CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}



#Customized stop words.
stop_words = {'a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 
              'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 
              'doing', 'don', 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'he', 'her', 'here', 
              'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in', 'into', 'it', "it's", 'its', 
              'itself', 'just', 'm', 'ma', 'me', 'more', 'most', 'my', 'myself', 'now', 'o', 'of', 'off', 'on', 
              'once', 'only', 'or', 'other', 'our',  'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 
              'shan', "shan't", 'she', "she's", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 
              'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', 'this', 'those', 'through', 'to', 
              'too', 'under', 'until', 'up', 've', 'very', 'we', 'while', 'with', 'y', 'you', "you'd", "you'll", 
              "you're", "you've", 'your', 'yours', 'yourself', 'yourselves'}



#Function for expanding contractions using above dictonary of contractions.
def expand_contractions(sentence, contraction_mapping):
    #Creating a list of contraction keys.
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags = re.IGNORECASE|re.DOTALL)
    
    #Function for expanding the contractions.
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = (contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower()))
        expanded_contraction = first_char + expanded_contraction[1:]
        return expanded_contraction
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence



#Function for creating a list ofstrings from the input. It creates different elements in the list if there is a space.
#Basically, a sentence will be converted into a list of words.
def words(text): 
  return re.findall(r'\w+', text.lower())

#Reading the file with vocabulary.
WORDS = Counter(words(open('/content/drive/My Drive/Colab_Datasets/Tweets_Data/big.txt').read()))

#Function for calculating probability of the given word to find in the vocabulary.
def P(word, N = sum(WORDS.values())): 
    "Probability of `word`."
    #WORDS[word] gives the count of word
    #N is total number of words in the vocabulary.
    return WORDS[word] / N

#Main function which should be called for word correction.
def correction(word): 
    "Most probable spelling correction for word."
    return max(candidates(word), key = P)

#Function for genetraing all possible real words from given string.
def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

#Function for editing given string. This returns words which are one edit away from input string.
#The function edits1 returns a set of all the edited strings (whether words or not) that can be made with one simple edit: 
#a simple edit to a word is a deletion (remove one letter), a transposition (swap two adjacent letters), a replacement (change one letter to another) or an insertion (add a letter).
#The output of this function can be a big set. For a word of length n, there will be n deletions, n-1 transpositions, 
#26n alterations, and 26(n+1) insertions, for a total of 54n+25 (of which a few are typically duplicates)
def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    #Splitting the letters of a word to form a combination.
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

#Function for generating set of words which are two edits away from input string.
def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))


#Function for converting a list to a string.
def listToString(inlist):
    #Initialize an empty string.
    string = " "
    return (string.join(inlist))




#Function for preprocessing and normalizing text data.
def data_normalization(indata):
    #Setting stop word variable using NLTK package.
    stopWords = set(stopwords.words('english'))
    
    #Creating list of punctuations.
    punctuations = [".", ",", "!", "`", "...", "****", ":", "(", ")", "?", "-", "{", "}", "[", "]", ";", "/", "+", "<", 
               ">", "|", "=" "_", "^", "@", "#", "$", "%", "~", "'", ".."]
    
    #Regular expression tokenizer with only alphanumeric contents.
    Token_Pattern = r'\w+'
    regex_wt = nltk.RegexpTokenizer(pattern = Token_Pattern, gaps = False)

    #Listing out all contractions.
    contaction_keys = list(CONTRACTION_MAP.keys())

    #Declaring list for storing lemmatized tweets.
    lemmatized_words = []
    for i in range(len(indata)):
        #Condition for empty tweets.
        if (str(indata[i]) == "nan"):
            #Fake strink in the blank tweet.
            indata[i] = "NA"
        #Expanding contractions.
        text_segment = indata[i].replace("`", "'")
        expanded_tweets = expand_contractions(text_segment, CONTRACTION_MAP)
        #Removing URL links from tweets.
        expanded_tweets = re.sub(r'http\S+', '', expanded_tweets, flags = re.MULTILINE)
        #print("After contractions: ", expanded_tweets)
        #Tokenizing the words.
        tokens = regex_wt.tokenize(expanded_tweets)
        #Declaring list for lemmatized words.
        lemm_list = []
        for word in tokens:
            #Condition for stopwords.
            #if (word not in stopWords):
            if (word not in stop_words):
                #Condition for punctuations.
                if (word not in punctuations):
                    #Correcting the word.
                    #Not using this function because its not correcting words properly. Need to find more accurate way!
                    #corrected_spell = correction(word)
                    corrected_spell = word
                    #Converting all letters to lower form.
                    corrected_spell = corrected_spell.lower()
                    #Lemmatizing words.
                    lemm_list.append(wordnet_lemmatizer.lemmatize(corrected_spell, pos = "v"))
        #Converting the lemmatized word's (document) list to a string.
        lemm_string = listToString(lemm_list)
        lemmatized_words.append(lemm_string)
    return lemmatized_words


#Function for extracting features from given data. There are three options of extraction methods are given:
#Those are Bag of Words-based frequencies, occurrences, and TF-IDF based features
def build_feature_matrix(documents, feature_type='frequency', ngram_range=(1, 1), min_df=0.0, max_df=1.0):
    feature_type = feature_type.lower().strip()
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary=True, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary=False, min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df=min_df, max_df=max_df, ngram_range=ngram_range)
    
    else:
        raise Exception("Wrong feature type entered. Possible values: 'binary', 'frequency', 'tfidf'")
    
    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    return (vectorizer, feature_matrix)


In [4]:
#Normalizing tweets.
normalized_tweets = data_normalization(raw_data['text'])

LookupError: ignored

In [22]:

print(raw_data['text'][0])
print(normalized_tweets[:2])


 I`d have responded, if I were going
['i would have respond i be go', 'sooo sad i will miss san diego']


In [24]:
documents = np.array(normalized_tweets[:2])
documents

sample_vectorizer, sample_feature_matrix = build_feature_matrix(documents, feature_type = 'tfidf', 
                                                                ngram_range = (1, 1), min_df = 0.0, max_df = 1.0)

#print(sample_vectorizer)
print(sample_feature_matrix[0])

array(['i would have respond i be go', 'sooo sad i will miss san diego'],
      dtype='<U30')

In [31]:

embedding_layer = Embedding(1000, 64)


In [51]:

#Number of words considered for constructing features.
max_features = 10000
#Curring off the text after given number of words.
maxlen = 20

#Loading data.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

#Converting the integers into a 2D integer tensor of shape (number of samples, max length)
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen = maxlen)


In [1]:

import os

imdb_dir = '/Users/Bhair/Downloads/aclImdb'
#C:\Users\Bhair\Downloads\aclImdb
train_dir = os.path.join(imdb_dir, 'train')
labels = []
texts = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join(train_dir, label_type)
  for fname in os.listdir(dir_name):
    if fname[-4:] == '.txt':
      f = open(os.path.join(dir_name, fname))
      texts.append(f.read())
      f.close()
      
      if label_type == 'neg':
        labels.append(0)
      else:
        labels.append(1)


FileNotFoundError: ignored